## import necessary items

In [1]:
import os 
import numpy as np
import pandas as pd
import random as rnd

## Loading and Preprocessing Dataset


In [2]:
data = pd.read_csv('../input/ner-dataset/ner_datasetreference.csv', encoding= 'unicode_escape')
data.head()

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O


In [3]:
data['Sentence #']=data['Sentence #'].ffill(axis = 0) 
data.head()

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,Sentence: 1,of,IN,O
2,Sentence: 1,demonstrators,NNS,O
3,Sentence: 1,have,VBP,O
4,Sentence: 1,marched,VBN,O


In [4]:
data_grouped = data.groupby(["Sentence #"])[['Word', 'POS', 'Tag']].agg(lambda x: list(x))
data_grouped.head()

,Word,POS,Tag
Sentence #,,,
Sentence: 1,"[Thousands, of, demonstrators, have, marched, ...","[NNS, IN, NNS, VBP, VBN, IN, NNP, TO, VB, DT, ...","[O, O, O, O, O, O, B-geo, O, O, O, O, O, B-geo..."
Sentence: 10,"[Iranian, officials, say, they, expect, to, ge...","[JJ, NNS, VBP, PRP, VBP, TO, VB, NN, TO, JJ, J...","[B-gpe, O, O, O, O, O, O, O, O, O, O, O, O, O,..."
Sentence: 100,"[Helicopter, gunships, Saturday, pounded, mili...","[NN, NNS, NNP, VBD, JJ, NNS, IN, DT, NNP, JJ, ...","[O, O, B-tim, O, O, O, O, O, B-geo, O, O, O, O..."
Sentence: 1000,"[They, left, after, a, tense, hour-long, stand...","[PRP, VBD, IN, DT, NN, JJ, NN, IN, NN, NNS, .]","[O, O, O, O, O, O, O, O, O, O, O]"
Sentence: 10000,"[U.N., relief, coordinator, Jan, Egeland, said...","[NNP, NN, NN, NNP, NNP, VBD, NNP, ,, NNP, ,, J...","[B-geo, O, O, B-per, I-per, O, B-tim, O, B-geo..."


In [5]:
sentences_list = data_grouped.Word.to_list()
pos_tags_list = data_grouped.POS.to_list()

In [9]:
max_len = max([len(s) for s in sentences_list])
print("Maximum length: ", max_len)

words=list(set(data['Word'].values))
words.sort()
vocab={word: i for i,word in enumerate(words)}
vocab["<PAD>"] = len(vocab)
vocab["<UNK>"] = len(vocab)
reverse_vocab = {v: k for k, v in vocab.items()}

tags=list(set(data['POS'].values))
tags.sort()
num_tags=len(tags)
tags_map={tag:i for i,tag in enumerate(tags)}
reverse_tag_map={v: k for k, v in tags_map.items()}

Maximum length:  104


In [10]:
def preprocess(vocab, tag_map, sentences, labels):
  sentences_int = []
  labels_int = []

  for sentence in sentences:
      s = [vocab[token] if token in vocab 
            else vocab['<UNK>']
            for token in sentence]
      sentences_int.append(s)

  for sentence in labels:
      l = [tag_map[label] for label in sentence] 
      labels_int.append(l) 
  return sentences_int, labels_int, len(sentences)

In [11]:
sentences_encoded, labels_encoded, data_length = preprocess(vocab, tags_map, sentences_list, pos_tags_list)

In [12]:
max_len=128
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

padded_sentences = pad_sequences(maxlen=max_len, sequences=sentences_encoded, padding="post", value=vocab["<PAD>"])
padded_tags = pad_sequences(maxlen=max_len, sequences=labels_encoded, padding="post", value=tags_map['$'])

print("Shape of padded sentences:", padded_sentences.shape)
print("Shape of padded labels:", padded_tags.shape)

Shape of padded sentences: (47959, 128)
Shape of padded labels: (47959, 128)


In [13]:
labels = [to_categorical(i, num_classes = num_tags) for i in  padded_tags]

## Splitting data into train, validation and test sets

In [14]:
from sklearn.model_selection import train_test_split
X_train, X_val_test, y_train, y_val_test = train_test_split(padded_sentences, labels, test_size = 0.3)
X_val, X_test, y_val, y_test = train_test_split(X_val_test, y_val_test, test_size = 0.3)
y_train = np.array(y_train)
y_test = np.array(y_test)
y_val = np.array(y_val)

print("X_train shape: ", X_train.shape)
print("y_train shape: ", y_train.shape)
print("X_test shape: ", X_test.shape)
print("y_test shape: ", y_test.shape)
print("X_val shape: ", X_val.shape)
print("y_val shape: ", y_val.shape)

X_train shape:  (33571, 128)
y_train shape:  (33571, 128, 42)
X_test shape:  (4317, 128)
y_test shape:  (4317, 128, 42)
X_val shape:  (10071, 128)
y_val shape:  (10071, 128, 42)


## Building Model

In [15]:
from tensorflow.keras import Model, Input
from tensorflow.keras.layers import LSTM, Embedding, Dense
from tensorflow.keras.layers import TimeDistributed, SpatialDropout1D, Bidirectional
from keras.layers.merge import add

In [16]:
embedding_dim = 64
vocab_size = len(vocab)
max_len = 128

input_layer = Input(shape = (max_len,))
embedding = Embedding(input_dim = vocab_size+1, output_dim = embedding_dim, input_length = max_len)(input_layer)
x = Bidirectional(LSTM(units=embedding_dim, return_sequences=True))(embedding)
x = Bidirectional(LSTM(units=embedding_dim, return_sequences=True))(x)
x = LSTM(units=embedding_dim, return_sequences=True)(x)
output = TimeDistributed(Dense(num_tags, activation="softmax"))(x)  #TimeDistributed layer applies Dense layer to each time stamp

model = Model(input_layer, output)
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=['categorical_accuracy'])
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 128)]             0         
_________________________________________________________________
embedding (Embedding)        (None, 128, 64)           2251584   
_________________________________________________________________
bidirectional (Bidirectional (None, 128, 128)          66048     
_________________________________________________________________
bidirectional_1 (Bidirection (None, 128, 128)          98816     
_________________________________________________________________
lstm_2 (LSTM)                (None, 128, 64)           49408     
_________________________________________________________________
time_distributed (TimeDistri (None, 128, 42)           2730      
Total params: 2,468,586
Trainable params: 2,468,586
Non-trainable params: 0
___________________________________________________

## Training Model

In [17]:
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), batch_size = 64, epochs = 10)

Epoch 1/10
525/525 [==============================] - 40s 55ms/step - loss: 0.7953 - categorical_accuracy: 0.8385 - val_loss: 0.4486 - val_categorical_accuracy: 0.8649
Epoch 2/10
525/525 [==============================] - 27s 51ms/step - loss: 0.3941 - categorical_accuracy: 0.8821 - val_loss: 0.1570 - val_categorical_accuracy: 0.9620
Epoch 3/10
525/525 [==============================] - 26s 50ms/step - loss: 0.1092 - categorical_accuracy: 0.9756 - val_loss: 0.0493 - val_categorical_accuracy: 0.9894
Epoch 4/10
525/525 [==============================] - 27s 51ms/step - loss: 0.0346 - categorical_accuracy: 0.9929 - val_loss: 0.0351 - val_categorical_accuracy: 0.9925
Epoch 5/10
525/525 [==============================] - 26s 50ms/step - loss: 0.0205 - categorical_accuracy: 0.9955 - val_loss: 0.0296 - val_categorical_accuracy: 0.9935
Epoch 6/10
525/525 [==============================] - 27s 51ms/step - loss: 0.0142 - categorical_accuracy: 0.9967 - val_loss: 0.0262 - val_categorical_accuracy:

## Saving Model

In [18]:
model.save('pos_model.h5')

## Loading saved model

In [19]:
from keras.models import load_model
model = load_model('pos_model.h5')

## Evaluating predictions using test set

In [20]:
y_pred = model.predict(X_test) 

In [21]:
def evaluate_prediction(y_pred, y_test, pad):
    outputs = np.argmax(y_pred, axis=2)
    labels = np.argmax(y_test, axis=2)
    mask = labels != pad
    accuracy = np.sum(outputs == labels) / float(np.sum(mask))
    return accuracy

In [22]:
acc = evaluate_prediction(y_pred, y_test, vocab["<PAD>"])
print('Accuracy on test set:', acc)

Accuracy on test set: 0.9946867037294418


## Prediction on custom input

In [23]:
def predict(sentence, model, vocab, tag_map):
    s = [vocab[token] if token in vocab else vocab['<UNK>'] for token in sentence.split(' ')]
    len_s = len(s)
    if len_s<128:
        padding = [vocab["<PAD>"]]*(128-len_s)
        s = s + padding
    else:
        s = s[:128]
    batch_data = np.ones((1, len(s)))
    batch_data[0][:] = s
    sentence = np.array(batch_data).astype(int)
    output = model(sentence)
    outputs = np.argmax(output, axis=2)
    labels = list(tag_map.keys())
    pred = []
    for i in range(len(outputs[0])):
        idx = outputs[0][i] 
        pred_label = labels[idx]
        pred.append(pred_label)
    return pred

In [24]:
sentence = "I am going to market for buying some foods"
predictions = predict(sentence, model, vocab, tags_map)
for x,y in zip(sentence.split(' '), predictions):
    if y != 'O':
        print(x, end="-->")
        print(y)

I-->PRP
am-->VBP
going-->VBG
to-->TO
market-->NN
for-->IN
buying-->VBG
some-->DT
foods-->NNS
